In [1]:
import pandas as pd
import os
import numpy as np

## Preparing from imputed data
The .npy have 97 days of data where the first 90 days are for training, and the remaining is for testing.
Notice that we need to use data in .npy to only fill the NaN values in the raw data

In [ ]:
root_path = './dataset/competition/missing_data_impu/'
flow_impu = np.load('./dataset/competition/missing_data_impu/flow-5min_imputed.npy')
speed_impu = np.load('./dataset/competition/missing_data_impu/speed-5min_imputed.npy')

flow_train_impu = np.round(flow_impu[:,:,:90], decimals=0)
K,D,L = flow_train_impu.shape
flow_train_impu = flow_train_impu.reshape(K,D*L).transpose(1,0)

flow_test_impu = np.round(flow_impu[:,:,90:], decimals=0)
K,D,L = flow_test_impu.shape
flow_test_impu = flow_test_impu.reshape(K,D*L).transpose(1,0)

speed_train_impu = np.round(speed_impu[:,:,:90], decimals=2)
K,D,L = speed_train_impu.shape
speed_train_impu = speed_train_impu.reshape(K,D*L).transpose(1,0)

speed_test_impu = np.round(speed_impu[:,:,90:], decimals=2)
K,D,L = speed_test_impu.shape
speed_test_impu = speed_test_impu.reshape(K,D*L).transpose(1,0)

In [ ]:
flow_train_impu.shape

#### Loading the raw data

In [ ]:
root_path = './dataset/competition/train-5min'
def load_train_raw(root_path):
    datetime_5min_train_ids = pd.date_range(start='2023-04-02', end='2023-07-01', freq='5min', inclusive='both')[:-1]

    # get a list of all the csv files in the directory
     # './dataset/train-30s'    './dataset/train-5min'
    file_list = [f for f in os.listdir(root_path) if f.endswith('.csv')]

    # sort the file list based on their name
    file_list = sorted(file_list, key=lambda x: int(x.split('.')[0]))
    flow_cols = ['Time', 'Lane 1 Flow (Veh/h)', 'Lane 2 Flow (Veh/h)', 'Lane 3 Flow (Veh/h)', 'Lane 4 Flow (Veh/h)']
    speed_cols = ['Time', 'Lane 1 Speed (km/h)', 'Lane 2 Speed (km/h)', 'Lane 3 Speed (km/h)', 'Lane 4 Speed (km/h)']
    occupy_cols = ['Time','Lane 1 Occ (%)','Lane 2 Occ (%)','Lane 3 Occ (%)','Lane 4 Occ (%)']

    flow_dfs = []
    speed_dfs = []

    # read each file one by one
    flag = 0
    for file in file_list:
        file_path = os.path.join(root_path, file)
        if flag == 1:
            flow_cols.remove('Time')
            speed_cols.remove('Time')
        flow_dfs.append(pd.read_csv(file_path).loc[:, flow_cols])
        speed_dfs.append(pd.read_csv(file_path).loc[:, speed_cols])
        flag += 1

    flow_train_df = pd.concat(flow_dfs,axis=1)
    speed_train_df = pd.concat(speed_dfs,axis=1)

    flow_train_df.columns = range(len(flow_train_df.columns))
    flow_train_df.rename(columns={0:'date'}, inplace=True)
    flow_train_df['date'] = datetime_5min_train_ids
    flow_train_df['date'] = pd.to_datetime(flow_train_df['date'])
    flow_train_df.set_index('date', inplace=True)


    speed_train_df.columns = range(len(speed_train_df.columns))
    speed_train_df.rename(columns={0:'date'}, inplace=True)
    speed_train_df['date'] = datetime_5min_train_ids
    speed_train_df['date'] = pd.to_datetime(speed_train_df['date'])
    speed_train_df.set_index('date', inplace=True)

    return flow_train_df, speed_train_df

flow_train_df, speed_train_df = load_train_raw(root_path)


In [ ]:
flow_train_arr = flow_train_df.values
mask = np.isnan(flow_train_arr)
flow_train_arr[mask] = flow_train_impu[mask]
flow_train_arr[flow_train_arr < 0] = np.nan

speed_train_arr = speed_train_df.values
mask = np.isnan(speed_train_arr)
speed_train_arr[mask] = speed_train_impu[mask]
speed_train_arr[speed_train_arr < 0] = np.nan

flow_train_df.iloc[:,:] = flow_train_arr
speed_train_df.iloc[:,:] = speed_train_arr

flow_train_df.fillna(method='bfill', inplace=True)
flow_train_df.fillna(method='ffill', inplace=True)
assert np.isnan(flow_train_df.values).sum() == 0
flow_train_df.to_csv(root_path+'/flow-5min.csv')

speed_train_df.fillna(method='bfill', inplace=True)
speed_train_df.fillna(method='ffill', inplace=True)
assert np.isnan(speed_train_df.values).sum() == 0
speed_train_df.to_csv(root_path+'/speed-5min.csv')

## Preparing from the raw data

### Preparing for training data

##### 5-min

In [3]:
datetime_5min_train_ids = pd.date_range(start='2023-04-02', end='2023-07-01', freq='5min', inclusive='both')[:-1]

# get a list of all the csv files in the directory
root_path = './dataset/competition/train-5min' # './dataset/train-30s'    './dataset/train-5min'
file_list = [f for f in os.listdir(root_path) if f.endswith('.csv')]

# sort the file list based on their name
file_list = sorted(file_list, key=lambda x: int(x.split('.')[0]))
flow_cols = ['Time', 'Lane 1 Flow (Veh/h)', 'Lane 2 Flow (Veh/h)', 'Lane 3 Flow (Veh/h)', 'Lane 4 Flow (Veh/h)']
speed_cols = ['Time', 'Lane 1 Speed (km/h)', 'Lane 2 Speed (km/h)', 'Lane 3 Speed (km/h)', 'Lane 4 Speed (km/h)']
occupy_cols = ['Time','Lane 1 Occ (%)','Lane 2 Occ (%)','Lane 3 Occ (%)','Lane 4 Occ (%)']

flow_dfs = []
speed_dfs = []
occupy_dfs = []

# read each file one by one
flag = 0
for file in file_list:
    file_path = os.path.join(root_path, file)
    if flag == 1:
        flow_cols.remove('Time')
        speed_cols.remove('Time')
        occupy_cols.remove('Time')
    flow_dfs.append(pd.read_csv(file_path).loc[:, flow_cols])
    speed_dfs.append(pd.read_csv(file_path).loc[:, speed_cols])
    occupy_dfs.append(pd.read_csv(file_path).loc[:, occupy_cols])
    flag += 1

flow_df = pd.concat(flow_dfs,axis=1)
speed_df = pd.concat(speed_dfs,axis=1)
occupy_df = pd.concat(occupy_dfs,axis=1)

flow_df.columns = range(len(flow_df.columns))
flow_df.rename(columns={0:'date'}, inplace=True)
flow_df['date'] = datetime_5min_train_ids
flow_df['date'] = pd.to_datetime(flow_df['date'])
flow_df.set_index('date', inplace=True)
flow_df.fillna(method='bfill', inplace=True)
flow_df.fillna(method='ffill', inplace=True)
flow_df.to_csv(root_path+'/flow-5min.csv')

speed_df.columns = range(len(speed_df.columns))
speed_df.rename(columns={0:'date'}, inplace=True)
speed_df['date'] = datetime_5min_train_ids
speed_df['date'] = pd.to_datetime(speed_df['date'])
speed_df.set_index('date', inplace=True)
speed_df.fillna(method='bfill', inplace=True)
speed_df.fillna(method='ffill', inplace=True)
speed_df.to_csv(root_path+'/speed-5min.csv')

occupy_df.columns = range(len(occupy_df.columns))
occupy_df.rename(columns={0:'date'}, inplace=True)
occupy_df['date'] = datetime_5min_train_ids
occupy_df['date'] = pd.to_datetime(occupy_df['date'])
occupy_df.set_index('date', inplace=True)
occupy_df.fillna(method='bfill', inplace=True)
occupy_df.fillna(method='ffill', inplace=True)
occupy_df.to_csv(root_path+'/occupy-5min.csv')

print('Number of NaN values in flow_df:', flow_df.isna().sum().sum())
print('Number of NaN values in speed_df:', speed_df.isna().sum().sum())

Number of NaN values in flow_df: 0
Number of NaN values in speed_df: 0


##### 30s Training data preparation

In [ ]:
# Define the start and end times of each range
ranges_30s = [('00:00:02', '23:59:34')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in ranges_30s:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='30s', inclusive='both').tolist()

# Create the DatetimeIndex from the list of values
datetime_30s_train_ids = pd.DatetimeIndex(index_values)

datetime_30s_train_ids

In [ ]:
datetime_30s_train_ids = pd.date_range(start='2023-04-02 00:00:02', end='2023-07-01 23:59:34', freq='30s', inclusive='both')[:-1]

# get a list of all the csv files in the directory
root_path = './dataset/train-30s' # './dataset/train-30s'    './dataset/train-5min'
file_list = [f for f in os.listdir(root_path) if f.endswith('.csv')]

# sort the file list based on their name
file_list = sorted(file_list, key=lambda x: int(x.split('.')[0]))
flow_cols = ['Time', 'Lane 1 Flow', 'Lane 2 Flow', 'Lane 3 Flow', 'Lane 4 Flow']
speed_cols = ['Time', 'Lane 1 G-Factor (22) Speed', 'Lane 2 G-Factor (22) Speed', 'Lane 3 G-Factor (22) Speed', 'Lane 4 G-Factor (22) Speed']
occupy_cols = ['Time','Lane 1 Occupancy (%)','Lane 2 Occupancy (%)','Lane 3 Occupancy (%)','Lane 4 Occupancy (%)']

flow_dfs = []
speed_dfs = []
occupy_dfs = []

# read each file one by one
flag = 0
for file in file_list:
    file_path = os.path.join(root_path, file)
    if flag == 1:
        flow_cols.remove('Time')
        speed_cols.remove('Time')
        occupy_cols.remove('Time')
    flow_dfs.append(pd.read_csv(file_path).loc[:, flow_cols])
    speed_dfs.append(pd.read_csv(file_path).loc[:, speed_cols])
    occupy_dfs.append(pd.read_csv(file_path).loc[:, occupy_cols])
    flag += 1

flow_df = pd.concat(flow_dfs,axis=1)
speed_df = pd.concat(speed_dfs,axis=1)
occupy_df = pd.concat(occupy_dfs,axis=1)

flow_df.columns = range(len(flow_df.columns))
flow_df.rename(columns={0:'date'}, inplace=True)
flow_df['date'] = datetime_30s_train_ids
flow_df['date'] = pd.to_datetime(flow_df['date'])
flow_df.set_index('date', inplace=True)
flow_df.fillna(0, inplace=True)
flow_df.to_csv(root_path+'/flow-30s.csv')

speed_df.columns = range(len(speed_df.columns))
speed_df.rename(columns={0:'date'}, inplace=True)
speed_df['date'] = datetime_30s_train_ids
speed_df['date'] = pd.to_datetime(speed_df['date'])
speed_df.set_index('date', inplace=True)
speed_df.fillna(0, inplace=True)
speed_df.to_csv(root_path+'/speed-30s.csv')

occupy_df.columns = range(len(occupy_df.columns))
occupy_df.rename(columns={0:'date'}, inplace=True)
occupy_df['date'] = datetime_30s_train_ids
occupy_df['date'] = pd.to_datetime(occupy_df['date'])
occupy_df.set_index('date', inplace=True)
occupy_df.fillna(0, inplace=True)
occupy_df.to_csv(root_path+'/occupy-30s.csv')

print('Number of NaN values in flow_df:', flow_df.isna().sum().sum())
print('Number of NaN values in speed_df:', speed_df.isna().sum().sum())

### Preparing for testing data (method1, not full-day data)

In [4]:
# Define the start and end times of each range
ranges_5min = [('05:00:00', '07:55:00'), ('09:30:00', '12:25:00'), ('14:00:00', '16:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

# Create the DatetimeIndex from the list of values
datetime_5min_test_ids = pd.DatetimeIndex(index_values)


# ranges_5min_empty = [('05:00:00', '08:55:00'), ('09:30:00', '13:25:00'), ('14:00:00', '17:55:00')]
ranges_5min_empty = [('05:00:00', '08:55:00'),('09:30:00', '13:25:00'), ('14:00:00', '17:55:00')]
# Create an empty list to store the DatetimeIndex values
index_values = []
# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in ranges_5min_empty:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

# Create the DatetimeIndex from the list of values
empty_df = pd.DatetimeIndex(index_values)
empty_df = pd.DataFrame(index=empty_df, columns=[str(i) for i in range(1, 41)])

# get a list of all the csv files in the directory
root_path = './dataset/competition/test-5min'
file_list = [f for f in os.listdir(root_path) if f.endswith('.csv')]

# sort the file list based on their name
file_list = sorted(file_list, key=lambda x: int(x.split('.')[0]))
flow_cols = ['Time', 'Lane 1 Flow (Veh/h)', 'Lane 2 Flow (Veh/h)', 'Lane 3 Flow (Veh/h)', 'Lane 4 Flow (Veh/h)']
speed_cols = ['Time', 'Lane 1 Speed (km/h)', 'Lane 2 Speed (km/h)', 'Lane 3 Speed (km/h)', 'Lane 4 Speed (km/h)']

flow_dfs = []
speed_dfs = []

# read each file one by one
flag = 0
for file in file_list:
    file_path = os.path.join(root_path, file)
    if flag == 1:
        flow_cols.remove('Time')
        speed_cols.remove('Time')
    flow_dfs.append(pd.read_csv(file_path).loc[:, flow_cols])
    speed_dfs.append(pd.read_csv(file_path).loc[:, speed_cols])
    flag += 1

flow_df = pd.concat(flow_dfs,axis=1)
speed_df = pd.concat(speed_dfs,axis=1)

flow_df.columns = range(len(flow_df.columns))
flow_df.rename(columns={0:'date'}, inplace=True)
flow_df['date'] = datetime_5min_test_ids
flow_df['date'] = pd.to_datetime(flow_df['date'])
flow_df.set_index('date', inplace=True)
flow_df.fillna(method='ffill', inplace=True)
flow_df.fillna(method='bfill', inplace=True)
merged = empty_df.merge(flow_df, how='left', left_index=True, right_index=True)
merged.dropna(axis=1, how='all', inplace=True)
merged.index.name = 'date'
merged.fillna(method='ffill', inplace=True)
merged.fillna(method='bfill', inplace=True)
merged.to_csv(root_path+'/flow-5min.csv')

speed_df.columns = range(len(speed_df.columns))
speed_df.rename(columns={0:'date'}, inplace=True)
speed_df['date'] = datetime_5min_test_ids
speed_df['date'] = pd.to_datetime(speed_df['date'])
speed_df.set_index('date', inplace=True)
speed_df.fillna(method='ffill', inplace=True)
speed_df.fillna(method='bfill', inplace=True)
merged = empty_df.merge(speed_df, how='left', left_index=True, right_index=True)
merged.dropna(axis=1, how='all', inplace=True)
merged.index.name = 'date'
merged.fillna(method='ffill', inplace=True)
merged.fillna(method='bfill', inplace=True)
merged.to_csv(root_path+'/speed-5min.csv')
# speed_df.to_csv(root_path+'/speed.csv')

print('Number of NaN values in flow_df:', flow_df.isna().sum().sum())
print('Number of NaN values in speed_df:', speed_df.isna().sum().sum())

Number of NaN values in flow_df: 0
Number of NaN values in speed_df: 0


### Preparing for testing data (method1, full-day data)

In [ ]:
# Define the start and end times of each range
ranges_5min = [('05:00:00', '07:55:00'), ('09:30:00', '12:25:00'), ('14:00:00', '16:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

# Create the DatetimeIndex from the list of values
datetime_5min_test_ids = pd.DatetimeIndex(index_values)

# Create the DatetimeIndex from the list of values
datetime_5min_test_ids_full = pd.date_range(start=start_date, end='2023-07-08', freq='5min')[:-1]
empty_df = pd.DataFrame(index=datetime_5min_test_ids_full, columns=[str(i) for i in range(1, 41)])

In [ ]:
# get a list of all the csv files in the directory
root_path = './dataset/test-5min'
file_list = [f for f in os.listdir(root_path) if f.endswith('.csv')]

# sort the file list based on their name
file_list = sorted(file_list, key=lambda x: int(x.split('.')[0]))
flow_cols = ['Time', 'Lane 1 Flow (Veh/h)', 'Lane 2 Flow (Veh/h)', 'Lane 3 Flow (Veh/h)', 'Lane 4 Flow (Veh/h)']
speed_cols = ['Time', 'Lane 1 Speed (km/h)', 'Lane 2 Speed (km/h)', 'Lane 3 Speed (km/h)', 'Lane 4 Speed (km/h)']

flow_dfs = []
speed_dfs = []

# read each file one by one
flag = 0
for file in file_list:
    file_path = os.path.join(root_path, file)
    if flag == 1:
        flow_cols.remove('Time')
        speed_cols.remove('Time')
    flow_dfs.append(pd.read_csv(file_path).loc[:, flow_cols])
    speed_dfs.append(pd.read_csv(file_path).loc[:, speed_cols])
    flag += 1

flow_df = pd.concat(flow_dfs,axis=1)
speed_df = pd.concat(speed_dfs,axis=1)

In [ ]:
flow_df.columns = range(len(flow_df.columns))
flow_df.rename(columns={0:'date'}, inplace=True)
flow_df['date'] = datetime_5min_test_ids
flow_df['date'] = pd.to_datetime(flow_df['date'])
flow_df.set_index('date', inplace=True)

merged = empty_df.merge(flow_df, how='left', left_index=True, right_index=True)
merged.dropna(axis=1, how='all', inplace=True)
merged.index.name = 'date'
merged.fillna(0, inplace=True)
merged.to_csv(root_path+'/flow.csv')

speed_df.columns = range(len(speed_df.columns))
speed_df.rename(columns={0:'date'}, inplace=True)
speed_df['date'] = datetime_5min_test_ids
speed_df['date'] = pd.to_datetime(speed_df['date'])
speed_df.set_index('date', inplace=True)

merged = empty_df.merge(speed_df, how='left', left_index=True, right_index=True)
merged.dropna(axis=1, how='all', inplace=True)
merged.index.name = 'date'
merged.fillna(0, inplace=True)
merged.to_csv(root_path+'/speed.csv')